In [26]:
spark.stop()

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, regexp_replace, split
from pyspark.sql import functions as F
# spark = SparkSession.builder.appName("Jupyter").getOrCreate()
#spark.sql("SET spark.sql.catalog.my_catalog.uri=http://192.168.56.1:8181")

from dotenv import load_dotenv
import os
load_dotenv()
storage_account = os.getenv("AZURE_STORAGE_ACCOUNT")
adls_client_id = os.getenv("ADLS_CLIENT_ID")
adls_client_secret = os.getenv("ADLS_CLIENT_SECRET")
adls_tenant_id = os.getenv("ADLS_TENANT_ID")
value = os.getenv('AZURE_DATA_LAKE_CONNECTION_STRING')
container_name = os.getenv('AZURE_CONTAINER_NAME')

spark = SparkSession.builder \
    .appName("AzureADLS") \
    .config("spark.jars", "/opt/spark/jars/hadoop-azure-3.3.2.jar,/opt/spark/jars/hadoop-azure-datalake-3.3.2.jar,/opt/spark/jars/azure-identity-1.14.2.jar,/opt/spark/jars/mssql-jdbc-12.8.1.jre11.jar,/opt/spark/jars/msal4j-1.18.0.jar") \
    .config("spark.hadoop.fs.abfss.impl", "org.apache.hadoop.fs.azurebfs.SecureAzureBlobFileSystem") \
    .config(f"spark.hadoop.fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth") \
    .config(f"spark.hadoop.fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider") \
    .config(f"spark.hadoop.fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", f"{adls_client_id}") \
    .config(f"spark.hadoop.fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", f"{adls_client_secret}") \
    .config(f"spark.hadoop.fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{adls_tenant_id}/oauth2/token") \
    .getOrCreate()


spark.conf.set("spark.hadoop.fs.azure.impl", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
spark.conf.set("fs.abfss.impl", "org.apache.hadoop.fs.azurebfs.SecureAzureBlobFileSystem")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
spark.conf.set("fs.azure.impl.disable.cache", "true")

spark

25/01/06 08:40:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
pip install -r requirements.txt


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from azure.identity import ClientSecretCredential
from azure.storage.blob import BlobServiceClient

# Test credentials
credential = ClientSecretCredential(
    tenant_id= adls_tenant_id,
    client_id= adls_client_id,
    client_secret= adls_client_secret
)

# Test connection
service_client = BlobServiceClient(
    account_url="https://myvisekendatalake.blob.core.windows.net",
    credential=credential
)

# List containers to verify connectivity
print([container.name for container in service_client.list_containers()])


['iceberg-metadata', 'myviseken-data-lake']


In [24]:
spark.conf.set("fs.azure.io.retry.max.retries", "20")  # Increase retry count
spark.conf.set("fs.azure.io.retry.backoff", "5000")   # Increase backoff time in ms
spark.conf.set("fs.azure.io.retry.max.attempts", "20") # Max retry attempts
spark.conf.set("fs.azure.dfs.socket.timeout", "30000") # 30 seconds socket timeout
spark.conf.set("fs.azure.read.request.timeout", "120000") # 120 seconds read request timeout
spark.conf.set("fs.azure.io.read.max.range.size", "67108864")  # Default: 64MB
spark.conf.set("fs.azure.io.read.range.buffer.size", "10485760")  # Default: 10MB
spark.conf.set("fs.azure.io.read.range.buffer.timeout", "60000")  # Default: 60 seconds

In [6]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# spark.sparkContext.setLogLevel("DEBUG")
# spark.conf.set("fs.azure.debug.logging", "true")
csv_file = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/raw/carlist/data_20250102170445.csv"
csv_test_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/raw/carlist/data_20250102170445.csv"

# schema = StructType([
#     StructField("listing_id", IntegerType(), True),
#     StructField("title", StringType(), True),
#     StructField("installment", StringType(), True),
#     StructField("year", IntegerType(), True),
#     StructField("variant", StringType(), True),
#     StructField("mileage", IntegerType(), True),
#     StructField("transmission", StringType(), True),
#     StructField("location", StringType(), True),
#     StructField("state", StringType(), True),
#     StructField("listing_image", StringType(), True),
#     StructField("price", StringType(), True),
#     StructField("url", StringType(), True),
#     StructField("image", StringType(), True) 
# ])

# # # Read the CSV
df = spark.read.option("multiline", "true").csv(csv_test_path, header=True)
# df = spark.read.format("csv").option("header", "true").load(csv_file)
# # Read the file as text
# # raw_data = spark.read.text(csv_test_path)

# # # Manually process each line to handle newline in the last column
# # processed_data = raw_data.rdd.map(lambda x: x[0].replace("\n", " "))

# # # Convert back to DataFrame
# # df = spark.read.csv(processed_data)

# # Show the result
# df.show()

# # Show the first few rows
df.show()
# print(csv_test_path)

+----------+--------------------+------------+----+-------+-------+------------+--------------+------------+--------------------+------+--------------------+--------------------+
|listing_id|               title| installment|year|variant|mileage|transmission|      location|       state|       listing_image| price|                 url|               image|
+----------+--------------------+------------+----+-------+-------+------------+--------------+------------+--------------------+------+--------------------+--------------------+
|  15774123|2012 Perodua Myvi...|RM 214/month|2012|     SE|  77500|   Automatic|Seri Kembangan|    Selangor|https://img1.icar...|   328|https://www.carli...|['https://img1.ic...|
|  15964938|2014 Perodua Myvi...|RM 249/month|2014|     SE|  57500|   Automatic|Seri Kembangan|    Selangor|https://img1.icar...|   289|https://www.carli...|['https://img1.ic...|
|  15531831|2018 Perodua Myvi...|RM 555/month|2018|     AV|  80118|   Automatic|   Setiawangsa|Kuala Lump

In [28]:
# connecting spark server to the azure sql database

server_name = os.getenv("SQL_SERVER")
database_name = os.getenv("DB_NAME")
username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
aad_user = os.getenv("AAD_USER")
aad_password = os.getenv("AAD_PASSWORD")

# jdbc connection string 
jdbc_url = f"jdbc:sqlserver://{server_name}.database.windows.net:1433;database={database_name};user={username}@{server_name};password={password};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
jdbc_url_aad = f"jdbc:sqlserver://myviseken.database.windows.net:1433;database=myviseken-db;user={aad_user};password={aad_password};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;authentication=ActiveDirectoryPassword"
df = spark.read.jdbc(jdbc_url_aad, "test_table", properties={
    "user": f"{aad_user}",
    "password": f"{aad_password}",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
})
df.show()
# print(jdbc_url)

Py4JJavaError: An error occurred while calling o453.jdbc.
: com.microsoft.sqlserver.jdbc.SQLServerException: Failed to load MSAL4J Java library for performing ActiveDirectoryPassword authentication.
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.getFedAuthToken(SQLServerConnection.java:6019)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.onFedAuthInfo(SQLServerConnection.java:5995)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.processFedAuthInfo(SQLServerConnection.java:5829)
	at com.microsoft.sqlserver.jdbc.TDSTokenHandler.onFedAuthInfo(tdsparser.java:335)
	at com.microsoft.sqlserver.jdbc.TDSParser.parse(tdsparser.java:130)
	at com.microsoft.sqlserver.jdbc.TDSParser.parse(tdsparser.java:42)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.sendLogon(SQLServerConnection.java:6888)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.logon(SQLServerConnection.java:5434)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection$LogonCommand.doExecute(SQLServerConnection.

In [46]:
# connecting spark server to the azure sql database

server_name = os.getenv("SQL_SERVER")
database_name = os.getenv("SQL_DB")
username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")

# jdbc connection string 
jdbc_url = f"jdbc:sqlserver://{server_name}.database.windows.net:1433;database={database_name};user={username}@{server_name};password={password};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"

properties = {
    "user": f"{username}",
    "password": f"{password}",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# SQL query to check and create table
check_and_create_query = """
IF NOT EXISTS (
    SELECT 1
    FROM sys.tables
    WHERE name = 'test_table'
)
BEGIN
    CREATE TABLE test_schema_name.test_table (
        column1 INT,
        column2 NVARCHAR(50),
        column3 DATETIME
    );
END;
"""
# Execute the query
connection = spark._sc._gateway.jvm.java.sql.DriverManager.getConnection(jdbc_url_aad, properties["user"], properties["password"],properties["driver"])
stmt = connection.createStatement()
stmt.execute(check_and_create_query)
stmt.close()
connection.close()

Py4JError: An error occurred while calling z:java.sql.DriverManager.getConnection. Trace:
py4j.Py4JException: Method getConnection([class java.lang.String, class java.lang.String, class java.lang.String, class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:342)
	at py4j.Gateway.invoke(Gateway.java:276)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)



In [45]:
import os
print(os.environ['SPARK_CLASSPATH'])


KeyError: 'SPARK_CLASSPATH'

In [29]:
from azure.storage.blob import BlobServiceClient
import pandas as pd

blob_service_client = BlobServiceClient.from_connection_string(value)

# Access the container and blob
container_name = os.getenv('AZURE_CONTAINER_NAME')
blob_name = "raw/carlist/data_20250102170445.csv"
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Download the blob content
stream = blob_client.download_blob().readall()

# Load into pandas DataFrame
import io
df = pd.read_csv(io.BytesIO(stream))
print(df.head())

   listing_id                                              title  \
0    15774123  2012 Perodua Myvi 1.5 SE Hatchback (MONTHLY RM...   
1    15964938  2014 Perodua Myvi 1.3 SE Hatchback (MUKA RM289...   
2    15531831                    2018 Perodua Myvi 1.5 Hatchback   
3    15642030                  2019 Perodua Myvi 1.5 H Hatchback   
4    15678245                  2018 Perodua Myvi 1.3 X Hatchback   

    installment  year variant  mileage transmission        location  \
0  RM 214/month  2012      SE    77500    Automatic  Seri Kembangan   
1  RM 249/month  2014      SE    57500    Automatic  Seri Kembangan   
2  RM 555/month  2018      AV    80118    Automatic     Setiawangsa   
3  RM 576/month  2019       H    57500    Automatic   Petaling Jaya   
4  RM 511/month  2018       X    85519    Automatic   Petaling Jaya   

          state                                      listing_image   price  \
0      Selangor  https://img1.icarcdn.com/32147751/main-m_used-...     328   
1      S

In [8]:
myvi_df = spark.createDataFrame(df)
myvi_df = myvi_df \
        .select('listing_id', 'year', 'state', 'price','url') \
        .count()
        # .orderBy(F.col('year')).desc() \
        # .show()
# myvi_df.count()


25/01/03 09:28:45 WARN TaskSetManager: Stage 4 contains a task of very large size (1775 KiB). The maximum recommended task size is 1000 KiB.
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o165.count

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
